<a href="https://colab.research.google.com/github/apchavezr/-Analisis_Grandes_Volumenes_Datos/blob/main/Ejercicio_PySpark_Seguridad_Datos_Salud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🏥 Seguridad y privacidad en el análisis de datos de salud usando PySpark
Este notebook simula el procesamiento seguro de datos personales de salud en una entidad pública. Se incluyen prácticas como anonimización, control de acceso por roles y cifrado simbólico.

## 🎯 Objetivo
Simular una estrategia básica de seguridad y privacidad de datos en salud, categorizando roles y aplicando técnicas de protección sobre datos sensibles.

In [1]:
# Paso 1: Inicializar sesión de Spark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("SeguridadDatosSalud").getOrCreate()

In [2]:
# Paso 2: Crear datos simulados de pacientes con información sensible
data = [
    ("123456789", "Carlos Gómez", 45, "Diabetes", "analista"),
    ("987654321", "Laura Díaz", 30, "Hipertensión", "auditor"),
    ("112233445", "Jorge Ruiz", 65, "Cáncer", "desarrollador"),
    ("223344556", "Ana Torres", 52, "Asma", "analista"),
]
columnas = ["id", "nombre", "edad", "diagnostico", "rol"]
df = spark.createDataFrame(data, columnas)
df.show()

+---------+------------+----+------------+-------------+
|       id|      nombre|edad| diagnostico|          rol|
+---------+------------+----+------------+-------------+
|123456789|Carlos Gómez|  45|    Diabetes|     analista|
|987654321|  Laura Díaz|  30|Hipertensión|      auditor|
|112233445|  Jorge Ruiz|  65|      Cáncer|desarrollador|
|223344556|  Ana Torres|  52|        Asma|     analista|
+---------+------------+----+------------+-------------+



## 🔐 Paso 3: Anonimizar los datos personales (seudonimización del ID y ocultar nombre)
Para proteger la identidad de los pacientes, se reemplazan sus identificaciones por hashes simbólicos y se ocultan los nombres.

In [3]:
from pyspark.sql.functions import sha2, concat_ws, lit
df_seguro = df.withColumn("id_hash", sha2(df.id, 256)) \
               .drop("id") \
               .withColumn("nombre", lit("[PROTEGIDO]"))
df_seguro.select("id_hash", "nombre", "edad", "diagnostico", "rol").show(truncate=False)

+----------------------------------------------------------------+-----------+----+------------+-------------+
|id_hash                                                         |nombre     |edad|diagnostico |rol          |
+----------------------------------------------------------------+-----------+----+------------+-------------+
|15e2b0d3c33891ebb0f1ef609ec419420c20e320ce94c65fbc8c3312448eb225|[PROTEGIDO]|45  |Diabetes    |analista     |
|8a9bcf1e51e812d0af8465a8dbcc9f741064bf0af3b3d08e6b0246437c19f7fb|[PROTEGIDO]|30  |Hipertensión|auditor      |
|8aef254db3e4b69baff72c2d932f20a51a5e08b6e91b196b12f98d8151d12121|[PROTEGIDO]|65  |Cáncer      |desarrollador|
|1d983d28d95dc1f0eae8a1e68dd366077b6f15344c2368d565ad0ef37edd40d3|[PROTEGIDO]|52  |Asma        |analista     |
+----------------------------------------------------------------+-----------+----+------------+-------------+



## 🔒 Paso 4: Control de acceso simulado por rol
Un ejemplo de acceso segmentado: mostrar solo cierta información dependiendo del rol del usuario conectado.

In [4]:
# Simulación: filtro para analistas (solo edad y diagnóstico)
rol_usuario = "analista"
if rol_usuario == "analista":
    df_seguro.select("edad", "diagnostico").show()
elif rol_usuario == "auditor":
    df_seguro.select("id_hash", "edad", "diagnostico").show()
elif rol_usuario == "desarrollador":
    df_seguro.select("rol").distinct().show()

+----+------------+
|edad| diagnostico|
+----+------------+
|  45|    Diabetes|
|  30|Hipertensión|
|  65|      Cáncer|
|  52|        Asma|
+----+------------+



## ✅ Conclusión
- Este ejercicio demuestra cómo aplicar principios de protección de datos personales en un entorno PySpark.
- Se aplicó seudonimización y restricciones de acceso por rol, respetando el principio de minimización.
- En contextos reales se complementaría con cifrado real (KMS), autenticación de usuarios y auditoría de accesos.